In [1]:
from quippy.descriptors import Descriptor
from ase.io import read
import numpy as np


In [2]:
Descriptor_description = f"soap cutoff=3.0 zeta=2 delta = 1.0 atom_sigma =1.0 l_max=3 n_max=5 n_Z=2 Z={{1 3}} n_species=2 species_Z={{1 3}}"

In [3]:
def _descr_deriv_atoms(self,molecules):
    soap = Descriptor(Descriptor_description)
    soaps = soap.calc(molecules, grad=True)
    soap_atoms=[]
    soap_derivatives=[]
    for mol in soaps:
        for at in mol["data"]:
            #norm = np.linalg.norm(at)
            soap_atoms.append(at)
        for at in mol["grad_data"]:
            soap_derivatives.append(at)

    return soap_atoms, soap_derivatives

In [4]:
def _an_kernel_gradient(self,ref_mol,zeta=2):
    if self.sparse == False:
        desc_train = self.training_set_descriptors
    elif self.sparse == True:
        desc_train = self.representing_set_descriptors
    desc1,der1 = self._descr_deriv_atoms([ref_mol],deriv)
    temp1 = []
    temp2 = []
    deriv_final = []
    for a in range(len(der1)):
        for b in range(3):
            for c in range(len(der1)):
                temp1.append(der1[c][a][b])
                #temp1.append(der1[a][c][b])
            temp2.append(temp1)
            temp1 = []
        deriv_final.append(temp2)
        temp2 = []
        normal_soap_vector = []
        norms = []
        for sv in desc1:
            norm = np.linalg.norm(sv)
            normal_soap_vector.append(sv/norm)
            norms.append(norm)  
        norms = np.array(norms)
    dKdr = [] 
    for iatom in range(len(ref_mol)):
        dKdri = []
        for direction in range(3):
            inter = deriv_final[iatom][direction] # derivation of atom in 3 direction based on all other atoms
            tt1 = (np.matmul(desc1,np.transpose(inter))) 
            vd = np.diag(tt1)
            tt2 = np.multiply(desc1,vd[:,None])
            r1 = tt2/norms[:,None] # each atomistic SOAP descriptor is normalized 
            f1 = np.multiply(inter,norms[:,None]) 
            norms2 = norms**2
            normDer=(f1-r1)/norms2[:,None]

            t1 = (np.matmul(normal_soap_vector,np.transpose(desc_train))**(zeta-1))
            t2 = (np.matmul(normDer,np.transpose(desc_train)))
            K = zeta*np.multiply(t1,t2)
            dKdri.append(K)
        dKdr.append(dKdri)
    return dKdr

In [5]:
all_molecules = read("/datavon1/DatabasesMolecules/LiH/LiH.xyz@:",format="extxyz")
last_one = all_molecules[-10:]

In [18]:
Descriptor_description = f"soap cutoff=5.0 covariance_type=dot_product zeta=2 delta=1 atom_sigma = 0.7 l_max=6 n_max=8 n_sparse=1000 sparse_method=cur_points n_Z=2 Z={{1 3}} n_species=2 species_Z={{1 3}}"
desdictGAP = f"soap cutoff=10.0 zeta=2 delta = 1.0 atom_sigma =1.0 l_max=3 n_max=5 n_Z=2 Z={{1 3}} n_species=2 species_Z={{1 3}}"
soap = Descriptor(desdictGAP)
soaps = soap.calc(last_one, grad=True)

In [23]:
print(list(soaps[0]))
print(len(last_one[0]))
print(soaps[0]['data'].shape)
print(soaps[0]['grad_data'].shape)
print(soaps[0]['grad_index_0based'].shape)

['has_grad_data', 'ii', 'pos', 'grad_covariance_cutoff', 'covariance_cutoff', 'data', 'has_data', 'grad_data', 'ci', 'grad_index_0based']
26
(26, 221)
(676, 3, 221)
(676, 2)


In [8]:
print(list(soaps[0]))
for a in (soaps[0]["grad_index_0based"]):
    print(a)

['has_grad_data', 'ii', 'pos', 'grad_covariance_cutoff', 'covariance_cutoff', 'data', 'has_data', 'grad_data', 'ci', 'grad_index_0based']
[0 0]
[0 9]
[0 3]
[0 1]
[1 1]
[1 0]
[ 1 10]
[1 4]
[1 2]
[2 2]
[2 1]
[ 2 11]
[2 5]
[3 3]
[3 0]
[ 3 12]
[3 6]
[3 4]
[4 4]
[4 3]
[4 1]
[ 4 13]
[4 7]
[4 5]
[5 5]
[5 4]
[5 2]
[5 8]
[6 6]
[6 3]
[ 6 15]
[6 7]
[7 7]
[7 6]
[7 4]
[ 7 16]
[7 8]
[8 8]
[8 7]
[8 5]
[ 8 17]
[9 9]
[9 0]
[ 9 18]
[ 9 12]
[ 9 10]
[10 10]
[10  9]
[10  1]
[10 19]
[10 13]
[10 11]
[11 11]
[11 10]
[11  2]
[11 20]
[11 14]
[12 12]
[12  9]
[12  3]
[12 21]
[12 15]
[12 13]
[13 13]
[13 12]
[13 10]
[13  4]
[13 16]
[13 14]
[14 14]
[14 13]
[14 11]
[14 23]
[14 20]
[14 17]
[15 15]
[15 12]
[15  6]
[15 24]
[15 16]
[16 16]
[16 15]
[16 13]
[16  7]
[16 25]
[16 17]
[17 17]
[17 16]
[17 14]
[17  8]
[18 18]
[18  9]
[18 21]
[18 19]
[19 19]
[19 18]
[19 10]
[19 22]
[19 20]
[20 20]
[20 19]
[20 14]
[20 11]
[20 23]
[21 21]
[21 18]
[21 12]
[21 24]
[21 22]
[22 22]
[22 21]
[22 19]
[22 25]
[22 24]
[22 23]
[23 23]
[23 22